# Creating a network from a file

In [1]:
from roseau.load_flow import *

## From JSON

The first format that can be read is the JSON format. The `from_json` class-method of `ElectricalNetwork` can be used
for that.

In [2]:
en = ElectricalNetwork.from_json("./test.json")

Note that you can save an `ElectricalNetwork` instance using the method `to_json` if you want.

Then, a load flow can be directly solved

In [3]:
en.solve_load_flow(auth=("username", "dev password"))

2

The results are accessible the same way as explained in the [Getting Started](Getting_Started.ipynb) page

In [4]:
buses_results, branches_results, loads_results = en.results
buses_results.head()

potential
bus_id      phase                        
MVLV08729   a      345.688349+196.010353j
            b     -4.326850-0396.2229050j
            c     -343.079200+201.149697j
            n      0.00000000+0.00000000j
LVBus079601 a      344.152441+194.816794j

In [5]:
branches_results.head()

current1               current2
branch_id      phase                                              
LVBranch085260 a       99.399958+19.838907j  -99.402324-19.834732j
               b     -48.112206-135.287714j  48.116986+135.287660j
               c      -57.311872+66.903227j   57.309441-66.907371j
               n      6.023850+048.5450850j -6.023844-048.5450830j
LVBranch159361 a       62.126379+12.614438j  -62.128237-12.611154j

Note that the buses, branches (lines, transformers and switches), and loads can be accessed from the electrical network
using the `buses`, `branches` and `loads` fields.

In [6]:
en.buses

{'MVLV08729': <roseau.load_flow.models.buses.Bus at 0x7f2afc459430>,
 'LVBus079601': <roseau.load_flow.models.buses.Bus at 0x7f2afc459400>,
 'LVBus079605': <roseau.load_flow.models.buses.Bus at 0x7f2afc3e0df0>,
 'LVBus079606': <roseau.load_flow.models.buses.Bus at 0x7f2afc3e0af0>,
 'LVBus079602': <roseau.load_flow.models.buses.Bus at 0x7f2abc68ec40>,
 'LVBus079604': <roseau.load_flow.models.buses.Bus at 0x7f2abc68ed60>,
 'LVBus079607': <roseau.load_flow.models.buses.Bus at 0x7f2abc68ef40>,
 'LVBus079615': <roseau.load_flow.models.buses.Bus at 0x7f2abc673070>,
 'LVBus079603': <roseau.load_flow.models.buses.Bus at 0x7f2abc673160>,
 'LVBus183333': <roseau.load_flow.models.buses.Bus at 0x7f2abc673250>,
 'LVBus183322': <roseau.load_flow.models.buses.Bus at 0x7f2abc673340>,
 'LVBus183352': <roseau.load_flow.models.buses.Bus at 0x7f2abc673430>,
 'LVBus183334': <roseau.load_flow.models.buses.Bus at 0x7f2abc673520>,
 'LVBus183323': <roseau.load_flow.models.buses.Bus at 0x7f2abc673610>,
 'LVBus1

## Adding elements

It is possible to add new buses, loads, lines, switches, transformers to an existing network using the `add_element`
method of the class `ElectricalNetwork`. For instance, the code below adds a new line and constant power load.

In [7]:
# Connect a new bus to the network
ground = en.special_elements[0]
bus = Bus(id="new_bus", n=4)
load = PowerLoad(id="new_load", n=4, bus=bus, s=[100, 0, 0])
lc = LineCharacteristics.from_name_lv("S_AL_240")
line = ShuntLine(
    id="new_line", n=4, bus1=en.buses["LVBus079605"], bus2=bus, ground=ground, line_characteristics=lc, length=0.5
)
en.add_element(bus)
en.add_element(load)
en.add_element(line)

Then, the load flow can be solved again, and the new results are available.

In [8]:
# Rerun the results
en.solve_load_flow(auth=("username", "dev password"))
buses_results, branches_results, loads_results = en.results
buses_results.head()

potential
bus_id      phase                        
MVLV08729   a      345.694097+195.993383j
            b     -4.326607-0396.2223020j
            c     -343.087259+201.148230j
            n      0.00000000+0.00000000j
LVBus079601 a      344.154807+194.791921j

In [9]:
branches_results.head()

current1               current2
branch_id      phase                                              
LVBranch085260 a       99.741804+20.128371j  -99.744170-20.124195j
               b     -48.128450-135.287579j  48.133230+135.287525j
               c      -57.203770+66.729289j   57.201339-66.733432j
               n      5.590058+048.4292720j -5.590052-048.4292700j
LVBranch159361 a       62.130195+12.611259j  -62.132053-12.607975j